In [30]:
import pandas as pd
import numpy as np
train_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/train.csv")
test_titanic = pd.read_csv("/home/tuandinh/Desktop/ML and AI/Spaceship Titanic competiton/datasets/test.csv")
train_titanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


### **Handle missing values**

In [31]:
# To split features Cabin to ["Deck", "Side"]
def deck_side_encode(data):
    split_cabin = pd.DataFrame(list(map(lambda x: str(x).split("/"), data.Cabin)))
    new_data = data.copy()
    new_data["Side"] = split_cabin[2]
    new_data["Deck"] = split_cabin[0]
    new_data = new_data.drop("Cabin", axis=1)
    return new_data

In [32]:
attribs_num = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
attribs_cat = ["HomePlanet", "Destination", "Cabin"]
attribs_cat_bool = ["CryoSleep", "VIP"]

In [33]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")
imputer_cat_bool = SimpleImputer(strategy="most_frequent")
train_titanic[attribs_num] = imputer.fit_transform(train_titanic[attribs_num])
train_titanic[attribs_cat_bool] = imputer_cat_bool.fit_transform(train_titanic[attribs_cat_bool])
train_titanic[attribs_cat] = imputer_cat_bool.fit_transform(train_titanic[attribs_cat])
train_titanic.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [34]:
train_titanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [35]:
train_titanic_encoder = deck_side_encode(train_titanic)
train_titanic_encoder.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Side,Deck
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,P,B
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,S,F
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,S,A
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,S,A
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,S,F


In [36]:
X_train = train_titanic_encoder.drop(["Transported"], axis=1)
y_train = train_titanic_encoder["Transported"]

# label encoder 
for column_name in X_train.select_dtypes("object"):
    X_train[column_name], _ = X_train[column_name].factorize()
y_train = y_train.astype(int)

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X_train.dtypes == int

X_train.head(), y_train.head()

(   PassengerId  HomePlanet  CryoSleep  Destination   Age  VIP  RoomService  \
 0            0           0          0            0  39.0    0          0.0   
 1            1           1          0            0  24.0    0        109.0   
 2            2           0          0            0  58.0    1         43.0   
 3            3           0          0            0  33.0    0          0.0   
 4            4           1          0            0  16.0    0        303.0   
 
    FoodCourt  ShoppingMall     Spa  VRDeck  Name  Side  Deck  
 0        0.0           0.0     0.0     0.0     0     0     0  
 1        9.0          25.0   549.0    44.0     1     1     1  
 2     3576.0           0.0  6715.0    49.0     2     1     2  
 3     1283.0         371.0  3329.0   193.0     3     1     2  
 4       70.0         151.0   565.0     2.0     4     1     1  ,
 0    0
 1    1
 2    0
 3    0
 4    1
 Name: Transported, dtype: int64)

In [37]:
from sklearn.feature_selection import mutual_info_regression 
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X,y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI_score", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X_train, y_train, discrete_features)
mi_scores[::3]

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

>+ **I try to fix it**